In [ ]:
from molsim import MolecularDynamics
import matplotlib.pyplot as plt
import numpy as np

<div style="max-width: 1000px; margin-left: 0; margin-right: auto; font-size: 20px; line-height: 1.6;">

# Exercise 3: Lennar-Jones Molecular Dynamics

The MD program consists of a main MolecularDynamics object, found in `src/molecularDynamics/md.h`. The object has a constructor, that allows you to set a bunch of different settings that you might want to use in this simulation. Furthermore you will find the thermostats in `src/molecularDynamics/thermostats.h` and some samplers in `src/molecularDynamics/samplers.h`.

### MolecularDynamics object
The `MolecularDynamics` struct has a bunch of different functions. First there is the constructor `MolecularDynamics::MolecularDynamics`, which, from the input settings, sets up the simulation. First it precomputes some properties, like volume and density. Secondly, it initializes the velocities according the Maxwell-Boltzmann distribution (note that because mass = 1, in this case and therefore momentum = velocity). 

It also initializes all positions on a cubic lattice by calling `MolecularDynamics::latticeInitialization`, which prevents overlaps from happening, but a further minimization to the closest local minimum is done by `MolecularDynamics::gradientDescent`.

### Force calculation and integration
In MD one propagates the system by integrating the equations of motion. The integrator used for the equations of motion is the velocity Verlet integrator, which is a symplectic integrator. This means that the Hamiltonian is conserved, or, in other words, the drift or error in the ideally constant Hamiltonian is bounded. The Hamiltonian for an NVE system is (settings $m=1$)

$\mathcal{H}(p, q) = \sum_i^{N} \frac{p_i^2}{2} + \mathcal{U}(q)$

yielding the equations of motion:

$\dot{q} = \frac{\partial \mathcal{H}}{\partial p} = \frac{p}{m}$

$\dot{p} = -\frac{\partial \mathcal{H}}{\partial q} = -\frac{\partial \mathcal{U}}{\partial q}$.

In the velocity verlet algorithm these equations of motion are governed by updating the velocity in two half steps, once with the forces at time $t$ and once with the forces at time $t+\Delta t$.

This leads to integration scheme:

$v(t + \frac{\Delta t}{2}) = v(t) + F(t) \frac{\Delta t}{2}$

$q(t + \Delta t) = q(t) + v(t) \Delta t + F(t) \frac{(\Delta t)^2}{2}$

$v(t + \Delta t) = v(t + \frac{\Delta t}{2}) + F(t + \Delta t) \frac{\Delta t}{2}$

where

$F(t) = -\frac{\partial \mathcal{U}(q(t))}{\partial q}$.

In the code this force term is calculated in the method `MolecularDynamics::calculateForce`. This is the most computationally demanding function as it is a loop over all particle pairs. It can be nicely combined with computing the potential energy and the virial, which is used in the pressure. All particles have interaction with eachother according to the so-called Lennard-Jones potential - which has a repulsive term $r^{-12}$, which can be seen as the Pauli repulsion and an attraction term $r^{-6}$, which is often called the van der Waals term, as the van der Waals interaction also diminishes with $r^{-6}$. 

$\mathcal{U}(q) = \frac{1}{2} \sum_i^N \sum_{j \neq i}^N U_{LJ}(r_{ij})$

where

$r_{ij} = |q_i - q_j|$

and 

$U_{LJ}(r_{ij}) = 4 \epsilon \left((\frac{\sigma}{r})^{12} - (\frac{\sigma}{r})^{6} \right)$.

This force is then used in `MolecularDynamics::integrate` to sequentially update the momenta, positions and forces in given the steps in the equations of motion. At the same time, it might also make calls to the thermostat, which allows us to run a constant temperature simulation.

### Thermostats
In this tutorial we use just NVE (constant number of particles N, volume V and energy E) simulations. However, during the equilibration the kinetic energy is kept constant. While we do a constant energy simulation, it is preferable to give an input temperature, as this is more natural when doing research. Early on there might still be large shifts in kinetic energy due to the equilibration and therefore we keep it constant, using a `VelocityScaling` thermostat. This thermostat simply rescales the velocities to exactly reproduce the input temperature and therefore does not conserve energy and therefore does not sample the NVE ensemble.

<div style="max-width: 1000px; margin-left: 0; margin-right: auto; font-size: 20px; line-height: 1.6;">

## Question 1
Run your first MD simulation with this program. 
In the cell under this cell you will find the constructor and the run methods of the MD package. Familiarize yourself with the method of setting this up and the different arguments given.

In [ ]:
# Initialize MD simulation
md = MolecularDynamics(
    numberOfParticles=200,
    temperature=1.0,
    dt=0.005,
    boxSize=8.0,
    numberOfEquilibrationSteps=int(1e4),
    numberOfProductionSteps=int(1e4),
    outputPDB=True,
    logLevel=0,
    seed=12,
    sampleFrequency=10,
)
# Run MD simulation
md.run()

<div style="max-width: 1000px; margin-left: 0; margin-right: auto; font-size: 20px; line-height: 1.6;">

Run this simulation with outputPDB set to `true`. This will generate a file named `movie.pdb` in the current directory. Let's view the movie:

In [ ]:
import nglview as nv
import MDAnalysis as mda

movie = mda.Universe("movie.pdb")
view = nv.show_mdanalysis(movie)
view.add_unitcell()
view.remove_ball_and_stick()
view.camera = "orthographic"
view.add_spacefill(radius=0.4, color="pink")
view

<div style="max-width: 1000px; margin-left: 0; margin-right: auto; font-size: 20px; line-height: 1.6;">

If the notebook NGLview-viewer does not work, then visualize this file with your favourite visualization software (ie. ovito, vmd, ...).

## Question 2
How is one able to control the temperature in this program? After all, the total energy of the system should be constant (not the temperature) and see how the kinetic energy scales with temperature.

Plot the `kineticEnergies` and `observedTemperatures` of two different input temperatures.

In [ ]:
# Initialize MD simulation
md_T2 = MolecularDynamics(
    numberOfParticles=200,
    temperature=2.0,
    dt=0.005,
    boxSize=8.0,
    numberOfEquilibrationSteps=int(1e4),
    numberOfProductionSteps=int(1e4),
    outputPDB=True,
    logLevel=0,
    seed=12,
    sampleFrequency=10,
)
# Run MD simulation
md_T2.run()

In [ ]:
# Plot results
fig, ax = plt.subplots(2, 1)

# start implementation
# end implementation

ax[0].set_xlabel(r"time / $\tau$")
ax[0].set_ylabel(r"E$_{kin}$ / $\varepsilon$")

ax[1].set_xlabel(r"time / $\tau$")
ax[1].set_ylabel(r"T / $\varepsilon$")

fig.tight_layout()

<div style="max-width: 1000px; margin-left: 0; margin-right: auto; font-size: 20px; line-height: 1.6;">

## Question 3
To test the energy drift $\Delta E$ of the numerical integration algorithm for a given time step $\Delta t$ after $N$ integration steps, one usually computes

\begin{equation}
\Delta E\left(\Delta t\right) = \frac{1}{N} \sum_{i=1}^{i=N} \left|
\frac{E\left(0\right) - E\left(i\Delta t\right)}{E\left(0\right)}
\right|
\end{equation}

In this equation, $E\left(t\right)$ is the total energy (kinetic+potential) of the system at time $t$. The program prints the average drift at the end of the run. To prevent drift, the timestep should be reasonably small given the system density. High density leads to larger forces, which lead to larger errors. Test to see how high you can push the timestep before the system starts to diverge. 

In [ ]:
# Plot results
fig, ax = plt.subplots(figsize=(8, 6))

# start implementation
# end implementation

ax.set_xlabel(r"Time, t / $\tau$")
ax.set_ylabel(r"Energy, E / $\varepsilon$")
ax.legend()

In [ ]:
# Plot results
fig, ax = plt.subplots(figsize=(8, 6))

# start implementation
# end implementation

ax.set_xlabel(r"Time, t / $\tau$")
ax.set_ylabel(r"Energy, E / $\varepsilon$")
ax.legend()

<div style="max-width: 1000px; margin-left: 0; margin-right: auto; font-size: 20px; line-height: 1.6;">

## Question 4
An important property of symplectic integrators is that they are time reversible. Can you run the simulation with a negative timestep? Do you expect a difference given the Equations of motion?

In [ ]:
# Initialize MD simulation
md_rev = MolecularDynamics(
    numberOfParticles=200,
    temperature=1.0,
    dt=-0.005,
    boxSize=8.0,
    numberOfEquilibrationSteps=int(1e4),
    numberOfProductionSteps=int(1e5),
    outputPDB=True,
    logLevel=0,
    seed=12,
    sampleFrequency=1000,
)
# Run MD simulation
md_rev.run()

In [ ]:
# Plot results
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(md_rev.time, md_rev.conservedEnergies, label="Conserved")

ax.set_xlabel(r"Time, t / $\tau$")
ax.set_ylabel(r"Energy, E / $\varepsilon$")
ax.legend()

<div style="max-width: 1000px; margin-left: 0; margin-right: auto; font-size: 20px; line-height: 1.6;">

## Question 5
The radial distribution function is an important tool to analyze the average local environment of particles. The ensemble average over all pairs of the pair distribution is defined as:

\begin{equation}
g(r) = \frac{2V}{N^2} \langle \sum_{i < j} \delta(r - r_{ij}) \rangle
\end{equation}

Generally, in a simulation, a delta function is not tractable, but a binning algorithm with a reasonable amount of bins will do. Which is also implemented in the `MolecularDynamics` program. In a liquid of spherically symmetric molecules $g(r)$ depends on the distance between the molecules. These definitions are useful since we can use them to give the probability of observing a molecule at r, given that we have a molecule at the origin. The function $\rho g(r)$ gives the local density of particles, given there is a molecule at the origin. Using the spherical volume element (and the spherical symmetry of the molecule that we have assumed) we can calculate the probability as follows:

\begin{equation}
\int_0^{\infty} \rho g(r) 4 \pi r^2 dr = N -1
\end{equation}

The integral states that there are N-1 particles outside the central particle we are observing. The ensemble average of any pair function may be expressed in this form

\begin{equation}
\langle \mathcal{A} \rangle = \frac{N}{2} \rho \int_0^\infty a(r) g(r) 4 \pi r^2 dr
\end{equation}

This can be used to calculate the average energy $\langle U \rangle$ or the potential of mean force (PMF) using the relation $w(r) = -k_BT \log(g(r))$. Another use is with Kirkwood-Buff integrals to calculate the activity coefficient or solubility.

One thing that is very immediately clear from a radial distribution function is the amount of order that can be found in the local environment. Generate the radial distribution for three different phases of the Lennard-Jones system: gas, liquid and solid. The MD program automatically samples the RDF, you can retrieve it by asking the sampler for results with `md.rdfSampler.getResults()`. This returns a $(N, 2)$ array, with pairs of values $(r, g(r))$ for $N$ different bins. 

What are the obvious differences between the RDF's of the three phases? Why is there no particle to be found in the region between 0 and approximately 1 $\sigma$?

**Hint**: Have a look at the phase diagram provided in Basic MC for the input settings.

In [ ]:
# Plot results
fig, ax = plt.subplots()

# get rdf and plot
# start implementation
# end implementation

ax.legend()
ax.set_xlabel(r"r / $\sigma$")
ax.set_ylabel("g(r)")

<div style="max-width: 1000px; margin-left: 0; margin-right: auto; font-size: 20px; line-height: 1.6;">

## Question 6
An important quantity of a liquid or gas is the so-called self-diffusion coefficient $D$. Here we calculate $D$ in a three-dimensional system. There are two methods to calculate $D$:

1.by integrating the velocity auto-correlation function (VACF):
\begin{equation}
D = \frac{1}{3}\int_{0}^{\infty} \left\langle {\bf
v}\left(t\right)\cdot {\bf v}\left(t+t^{'}\right) \right\rangle
dt^{'}\nonumber\\ 
= \frac{\int_{0}^{\infty} \sum_{i=1}^{i=N}
\left\langle{\bf v}\left(i,t\right)\cdot{\bf v}\left(i,t +
t^{'}\right)\right\rangle dt^{'}}{3N}
\end{equation}
in which $N$ is the number of particles and ${\bf v}\left(i,t\right)$ is the velocity of particle $i$ at time $t$. One should choose $t$ in such a way that independent time origins are taken, \emph{i.e.} $t = ia\Delta t$, $i=1,2, \cdots , \infty$ and $\left\langle {\bf v}\left(t\right)\cdot{\bf v}\left(t+a\Delta t\right)\right\rangle \approx {\bf 0}$ 


2.by calculating the mean-squared displacement:
\begin{equation}
D = \lim_{t^{'} \rightarrow \infty} \frac{\left\langle \left|{\bf
x}\left(t + t^{'}\right) - {\bf x} \left(t\right)\right|^{2}
\right\rangle}{6t^{'}}
\end{equation}

One should be careful with the mean-squared displacement calculation when periodic boundary conditions are used. Why? Calculate self-diffusion coefficientusing both methods.

Hint:The MD program automatically samples the MSD, you can retrieve it by asking the sampler for results with `md.msdSampler.getResults()`, in which normalizedMSD[i] = meanSquareDisplacements[i] / (numberOfParticles * sampleCounts[i]); normalizedVACF[i] = velocityAutocorrelation[i] / (numberOfParticles * sampleCounts[i]).

In [ ]:
msd = md.msdSampler.getResults()
fig, ax = plt.subplots()
t = msd[:, 0]
normalizedMSD = msd[:, 1]
normalizedVACF = msd[:, 3]
# start implementation
d_msd = 
d_vacf = 
# end implementation
ax.plot(t,d_msd, label=r"MSD")
ax.plot(t,d_vacf, label=r"VACF")
ax.legend()
ax.set_xlabel("t")
ax.set_ylabel("D")